In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('diabetes.csv')
X = df.drop('Outcome', axis=1).values  
y = df['Outcome'].values  

print(X)
print(y)

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]
[1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1
 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0
 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0

In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [11]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
       
        self.W1 = np.random.randn(input_size, hidden_size1)  
        self.b1 = np.zeros((1, hidden_size1)) 
        self.W2 = np.random.randn(hidden_size1, hidden_size2)  
        self.b2 = np.zeros((1, hidden_size2)) 
        self.W3 = np.random.randn(hidden_size2, output_size)  
        self.b3 = np.zeros((1, output_size))  

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def forward(self, X):
        
        self.z1 = np.dot(X, self.W1) + self.b1 
        self.a1 = self.sigmoid(self.z1)  
        self.z2 = np.dot(self.a1, self.W2) + self.b2  
        self.a2 = self.sigmoid(self.z2) 
        self.z3 = np.dot(self.a2, self.W3) + self.b3  
        self.a3 = self.sigmoid(self.z3) 
        return self.a3

    def backward(self, X, y, learning_rate):
       
        m = X.shape[0]  

        dZ3 = self.a3 - y
        dW3 = np.dot(self.a2.T, dZ3) / m
        db3 = np.sum(dZ3, axis=0, keepdims=True) / m

        dZ2 = np.dot(dZ3, self.W3.T) * self.sigmoid_derivative(self.z2)
        dW2 = np.dot(self.a1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dZ1 = np.dot(dZ2, self.W2.T) * self.sigmoid_derivative(self.z1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        self.W3 -= learning_rate * dW3
        self.b3 -= learning_rate * db3
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
           
            output = self.forward(X)

           
            self.backward(X, y, learning_rate)

            
            if epoch % 100 == 0:
                loss = self.compute_loss(y, output)
                print(f"Epoch {epoch}, Loss: {loss}")

    def compute_loss(self, y, y_pred):
    
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def predict(self, X):
        
        predictions = self.forward(X)
        return np.round(predictions)

In [13]:
input_size = X_train.shape[1]  
hidden_size1 = 10  
hidden_size2 = 5
output_size = 1  

nn = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)


nn.train(X_train, y_train, epochs=1000, learning_rate=0.01)

/tmp/ipykernel_10049/1530831681.py:12: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Epoch 0, Loss: 0.7560217493850326
Epoch 100, Loss: 0.6796879308683794
Epoch 200, Loss: 0.6561842542421655
Epoch 300, Loss: 0.6481633013419722
Epoch 400, Loss: 0.6452938283874508
Epoch 500, Loss: 0.6441917440052075
Epoch 600, Loss: 0.6437092723664355
Epoch 700, Loss: 0.6434480551077307
Epoch 800, Loss: 0.6432700416635168
Epoch 900, Loss: 0.6431275892637339


In [14]:
y_pred = nn.predict(X_test)


accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 64.29%
